In [6]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import warnings
warnings.filterwarnings("ignore")

In [7]:
def create_features(data):
    data = data.copy()
    data['pernah_dihubungi_sebelumnya'] = np.where(data['hari_sejak_kontak_sebelumnya'] == 999, 0, 1)
    data['rasio_sukses_sebelumnya'] = (
        data['hasil_kampanye_sebelumnya'].map({'success': 1, 'failure': 0, 'nonexistent': 0}) * (data['jumlah_kontak_sebelumnya'] + 1)
    )
    data['risk_score'] = (
        0.4 * data['indeks_harga_konsumen'] +
        0.3 * data['suku_bunga_euribor_3bln'] +
        0.3 * data['tingkat_variasi_pekerjaan']
    )
    data['usia_group'] = pd.cut(data['usia'], bins=[18, 30, 45, 60, 100], labels=['muda', 'paruhbaya', 'senior', 'lansia'])
    data['beban_pinjaman'] = (
        data['pinjaman_rumah'].map({'yes': 1, 'no': 0, 'unknown': 0}) +
        data['pinjaman_pribadi'].map({'yes': 1, 'no': 0, 'unknown': 0})
    )
    return data


In [8]:
df = pd.read_csv("training_dataset.csv")
df = create_features(df)

target = "berlangganan_deposito"
features = [col for col in df.columns if col not in [target, "customer_number"]]
X = df[features]
y = df[target]

# === Preprocessing ===
NUM_FEATURES = ['usia', 'jumlah_kontak_kampanye_ini', 'indeks_harga_konsumen',
                'suku_bunga_euribor_3bln', 'rasio_sukses_sebelumnya', 'risk_score']
CAT_FEATURES = ['pekerjaan', 'status_perkawinan', 'pendidikan', 'jenis_kontak',
                'pulau', 'usia_group', 'pernah_dihubungi_sebelumnya']

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), NUM_FEATURES),
    ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES)
])


In [9]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 100, 500),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
        'subsample': trial.suggest_float("subsample", 0.5, 1.0),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'scale_pos_weight': trial.suggest_float("scale_pos_weight", 1, 10),
        'random_state': 42,
        'tree_method': 'hist',
        'eval_metric': 'auc',
        'use_label_encoder': False,
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        X_train_transformed = preprocessor.fit_transform(X_train)
        X_valid_transformed = preprocessor.transform(X_valid)

        smote = SMOTE(random_state=42)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train_transformed, y_train)

        clf = XGBClassifier(**params)
        clf.fit(X_train_balanced, y_train_balanced)

        y_pred = clf.predict_proba(X_valid_transformed)[:, 1]
        aucs.append(roc_auc_score(y_valid, y_pred))

    return np.mean(aucs)


In [10]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)
print("Best ROC-AUC:", study.best_value)
print("Best Parameters:", study.best_params)


[I 2025-05-26 13:19:47,085] A new study created in memory with name: no-name-85f41cac-1ca2-4eef-99d1-f35330c7641f
[I 2025-05-26 13:19:55,130] Trial 0 finished with value: 0.7885437290934492 and parameters: {'n_estimators': 126, 'max_depth': 6, 'learning_rate': 0.011386829598181, 'subsample': 0.5560132900868923, 'colsample_bytree': 0.7940764034109309, 'scale_pos_weight': 5.851346176980327}. Best is trial 0 with value: 0.7885437290934492.
[I 2025-05-26 13:20:07,228] Trial 1 finished with value: 0.78708924473009 and parameters: {'n_estimators': 165, 'max_depth': 8, 'learning_rate': 0.023588333381243415, 'subsample': 0.6469555662168485, 'colsample_bytree': 0.7827697930069406, 'scale_pos_weight': 7.410796654665985}. Best is trial 0 with value: 0.7885437290934492.
[I 2025-05-26 13:20:11,820] Trial 2 finished with value: 0.7879164208638029 and parameters: {'n_estimators': 112, 'max_depth': 5, 'learning_rate': 0.13349305379991963, 'subsample': 0.6870767050812765, 'colsample_bytree': 0.69236899

Best ROC-AUC: 0.7898221974873179
Best Parameters: {'n_estimators': 176, 'max_depth': 9, 'learning_rate': 0.015598815605579434, 'subsample': 0.7881282974496064, 'colsample_bytree': 0.8148056398577201, 'scale_pos_weight': 1.3048438412205332}


In [11]:
best_params = study.best_params
best_params.update({'tree_method': 'hist', 'use_label_encoder': False, 'random_state': 42, 'eval_metric': 'auc'})

final_model = ImbPipeline([
    ("preprocess", preprocessor),
    ("smote", SMOTE(random_state=42)),
    ("clf", XGBClassifier(**best_params))
])
final_model.fit(X, y)   

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['usia',
                                                   'jumlah_kontak_kampanye_ini',
                                                   'indeks_harga_konsumen',
                                                   'suku_bunga_euribor_3bln',
                                                   'rasio_sukses_sebelumnya',
                                                   'risk_score']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['pekerjaan',
                                                   'status_perkawinan',
                                                   'pendidikan', 'jenis_kontak',
                                                   'pulau', 'usia_group',
                                                   'pernah_...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.015598815605579434, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=9,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=176,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=42, ...))])

In [12]:
val = pd.read_csv("validation_set.csv")
val = create_features(val)
X_val = val[features]
y_pred_prob = final_model.predict_proba(X_val)[:, 1]

if "berlangganan_deposito" in val.columns:
    y_val = val["berlangganan_deposito"]
    fpr, tpr, _ = roc_curve(y_val, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.4f}")
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    plt.figure(figsize=(6, 4))
    plt.plot(recall, precision, label="PR Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.grid(True)
    plt.show()
else:
    print("Kolom 'berlangganan_deposito' tidak tersedia pada validation set. Melewati evaluasi ROC/PR.")

Kolom 'berlangganan_deposito' tidak tersedia pada validation set. Melewati evaluasi ROC/PR.


In [13]:
submission = pd.DataFrame({
    "customer_number": val["customer_number"],
    "berlangganan_deposito": y_pred_prob
})
submission.to_csv("submission.csv", index=False)